### Combines CSV files to one DF

In [35]:
import  pandas as pd

# Read CSV file into DataFrame
df1 = pd.read_csv('saahavainnot_2019_kumpula.csv', sep=',')
df2 =  pd.read_csv('saahavainnot_2020_kumpula.csv', sep=',')
df3 =  pd.read_csv('saahavainnot_2021_kumpula.csv', sep=',')
df4 =  pd.read_csv('saahavainnot_2022_kumpula.csv', sep=',')


# Combines two csv files into one df. This way we can combine many csv files into one df
df = pd.concat([df1, df2, df3, df4])
# Get row count using len() function
row_count = len(df)
print(row_count)

df.head(1)

35068


,Vuosi,Kk,Pv,Klo,Aikavyöhyke,Sademäärä (mm),Sateen intensiteetti (mm/h),Lumensyvyys (cm),Ilman lämpötila (degC),Näkyvyys (m),Tuulen nopeus (m/s)
0,2019,1,1,00:00,UTC,0.9,1.1,7,0.8,3180,7.9


In [36]:
# Remove all rows with null values
df.dropna(inplace=True, axis=0)
# Remove all columns with null values
df.dropna(inplace=True, axis=1) 

# Get row count using len() function after removal of null values
row_count = len(df)
print(row_count)

# Mikäli edelleen sama, niin ei ole null arvoja
df.head(1)

35068


,Vuosi,Kk,Pv,Klo,Aikavyöhyke,Sademäärä (mm),Sateen intensiteetti (mm/h),Lumensyvyys (cm),Ilman lämpötila (degC),Näkyvyys (m),Tuulen nopeus (m/s)
0,2019,1,1,00:00,UTC,0.9,1.1,7,0.8,3180,7.9


In [37]:
#Creating PVM column
df['Kk'] = df['Kk'].apply(lambda x: '{0:0>2}'.format(x))
df['Pv'] = df['Pv'].apply(lambda x: '{0:0>2}'.format(x))
df['pvm'] = df['Vuosi'].astype(str) + df['Kk'].astype(str) + df['Pv'].astype(str) 
df.head(3)

,Vuosi,Kk,Pv,Klo,Aikavyöhyke,Sademäärä (mm),Sateen intensiteetti (mm/h),Lumensyvyys (cm),Ilman lämpötila (degC),Näkyvyys (m),Tuulen nopeus (m/s),pvm
0,2019,01,01,00:00,UTC,0.9,1.1,7,0.8,3180,7.9,20190101
1,2019,01,01,01:00,UTC,2.2,2.4,7,1.5,2360,8.5,20190101
2,2019,01,01,02:00,UTC,1.7,0.8,7,2,2800,9.8,20190101


In [38]:
# Generating new DF
newDf = df[['Sademäärä (mm)', 'Sateen intensiteetti (mm/h)', 'Lumensyvyys (cm)', 'Ilman lämpötila (degC)', 'Näkyvyys (m)', 'Tuulen nopeus (m/s)', 'pvm','Klo']].copy()
newDf.head(2)

,Sademäärä (mm),Sateen intensiteetti (mm/h),Lumensyvyys (cm),Ilman lämpötila (degC),Näkyvyys (m),Tuulen nopeus (m/s),pvm,Klo
0,0.9,1.1,7,0.8,3180,7.9,20190101,00:00
1,2.2,2.4,7,1.5,2360,8.5,20190101,01:00


In [39]:
# Rename columns 
newDf.rename(columns={'Sademäärä (mm)': 'sademaara', 'Sateen intensiteetti (mm/h)' : 'sateen_intensiteetti'}, inplace=True)
newDf.rename(columns={'Lumensyvyys (cm)': 'lumen_syvyys', 'Ilman lämpötila (degC)' : 'ilman_lampotila'}, inplace=True)
newDf.rename(columns={'Näkyvyys (m)': 'nakyvyys', 'Tuulen nopeus (m/s)' : 'tuulen_nopeus'}, inplace=True)
newDf.rename(columns={'Klo': 'klo'}, inplace=True)
newDf['klo'] = newDf['klo'].str.replace(':00', '')
newDf['sijainti_id'] = 154

# replaces values that have only - with 0
newDf.replace('-', 0, inplace=True)

newDf.head(3)

,sademaara,sateen_intensiteetti,lumen_syvyys,ilman_lampotila,nakyvyys,tuulen_nopeus,pvm,klo,sijainti_id
0,0.9,1.1,7,0.8,3180,7.9,20190101,00,154
1,2.2,2.4,7,1.5,2360,8.5,20190101,01,154
2,1.7,0.8,7,2,2800,9.8,20190101,02,154


### Writing New Dataframe to CSV

In [40]:
newDf.to_csv('FactSaa.csv', sep=',')

In [41]:
row_count = len(newDf)
print(row_count)
# 35068

35068


### Configurations / SQL Database etc.

In [42]:
import pyodbc
import pandas as pd
from warnings import simplefilter

SERVER = 'localhost'
DATABASE = 'FinalProject'
USER = 'sa'
PWD = 'Winter11'

simplefilter(action='ignore', category=UserWarning)

# Create a connection object
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)

# Create a cursor object
cursor = cnxn.cursor()


### Create FactSaa table if not exist

In [44]:

# Create a table if it doesn't exist
try:
    cursor.execute("""
    CREATE TABLE FactSaa(
        saa_id int IDENTITY(1,1) Primary Key,
        sademaara float,
        sateen_intensiteetti float,
        lumen_syvyys int,
        ilman_lampotila float,
        nakyvyys int,
        tuulen_nopeus float,
        pvm int,
        klo int,
        sijainti_id int
        )
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE FactSaa")

cnxn.commit()

# klo

### Load data into FactSaa table from newDf (data frame)

In [45]:
for index, row in newDf.iterrows():
    cursor.execute("INSERT INTO FactSaa values(?,?,?,?,?,?,?,?,?)",
                   float(row['sademaara']),
                   row['sateen_intensiteetti'],
                   row['lumen_syvyys'],
                   row['ilman_lampotila'],
                   row['nakyvyys'],
                   row['tuulen_nopeus'],
                   row['pvm'],
                   row['klo'],
                   row['sijainti_id']
                   )
    
cnxn.commit()


In [46]:
cursor.close()
cnxn.close()